In [4]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, AffectedPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',AffectedPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'({affected/clients:.2%})')
print('fecha reporte:',curr.report_date.strftime("%d/%m/%Y %H:%M"))

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=0))
res

usuarios regionales: 780,563 clientes
usuarios sin servicio : 1,026  clientes (0.13%)
fecha reporte: 21/08/2024 09:14


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
8,Viña del Mar,CHILQUINTA,2024-02-19,513,185 days 09:15:02.658611
9,Viña del Mar,CHILQUINTA,2024-08-21,230,1 days 09:15:02.658611
4,Quilpue,CHILQUINTA,2024-08-21,118,1 days 09:15:02.658611
3,Quilpue,CHILQUINTA,2024-02-19,83,185 days 09:15:02.658611
0,Cartagena,CHILQUINTA,2024-08-20,56,2 days 09:15:02.658611
6,San Antonio,CHILQUINTA,2024-08-20,19,2 days 09:15:02.658611
1,La Ligua,CGE,2024-08-21,2,1 days 09:15:02.658611
2,Puchuncavi,CGE,2024-08-21,2,1 days 09:15:02.658611
7,Viña del Mar,CGE,2024-08-21,2,1 days 09:15:02.658611
5,San Antonio,CGE,2024-08-21,1,1 days 09:15:02.658611


In [ ]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city